
<font size="5"><center><b>  CREDIT RISK ESTIMATION WITH MACHINE LEARNING AND ARTIFICIAL NEURAL NETWORKS ALGORITHMS </b></center></font>

<center><img src="https://i.hizliresim.com/4mmw6x6.png" style="width:30%;height:10%;"></center>

<center>One of the most important services of banks that attract their customers is credit. If  the prices of customers special needs has high prices, customers usually pay by credit. For this reason, many credit applications are made to banks. Banks compare various  information of customers while giving credit. As a result of this comparison, it gives credit  to the customer or not. Banks can give credit to customers in the long term. If the total  repayment amount of the credit given in the long term is too high with the interest,  sometimes customers cannot pay back. This situation causes the bank to money loss. Therefore, banks must sure that customers can repay their credit. </center>



## Features
<center>This dataset is available on the kaggle platform. The aim is to determine the risk of a bank credit given to customers. The risk status of the customers for the new credit will be determined by using some machine learning, artificial neural network model with this dataset. The dataset includes the following features:</center>

<p> <b>Age:</b> This variable bank contains the age information of the customers.</p>
<p> <b>Sex:</b> This variable contains the gender information of the bank customers.</p>
<p> <b>Job:</b> This variable contains occupational information of bank customers. Professions are divided into four categories according to skill level. (0 - unskilled and nonresident, 1 - unskilled and resident, 2 - skilled, 3 - highly skilled)</p>
<p> <b>Housing:</b> This variable includes the housing status of the bank customers.</p>
<p> <b>Saving Accounts:</b> This variable bank contains customers savings information.Customers savings are classified as little, moderate, quite rich, rich.</p>
<p> <b>Checking Accounts:</b> This variable show the cash in the accounts of bank customers.</p>
<p> <b>Credit Amount:</b> This variable shows how much credit the bank customers get. 
<p> <b>DM :Dustsch Mark</b> is used as currency.</p>
<p> <b>Duration:</b> This variable contains the specified time to pay the credit.</p>
<p> <b>Purpose:</b> This variable shows the purpose for which the bank customer taking  credit. Taking credit by customers are classified as car, furniture / equipment, radio / TV, domestic appliances, repairs, education, business, vacation / others.</p>
<p> <b>Risk:</b> This variable contains the customers risk information. Customers in the Data  Set have previously taken credit from banks. A value of 1 was given for customers who paid the credit and 0 for customers who could not.</p>

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import plotly.offline as py 
py.init_notebook_mode(connected=True)                  
import plotly.graph_objs as go                         
import plotly.tools as tls                             
from collections import Counter                        
import plotly.figure_factory as ff

from sklearn import preprocessing
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV                                         # to split the data
from sklearn.metrics import mean_squared_error, roc_auc_score, roc_curve, r2_score, accuracy_score, confusion_matrix, classification_report, fbeta_score     # to evaluate our model
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense

import warnings
import missingno as msno
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore")

pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
df = pd.read_csv('../input/german-credit-data-with-risk/german_credit_data.csv', index_col=0)

# 1. EDA (Exploratory of Data Analysis)
## 1.1. Data Preperation

In [ ]:
df.head()

In [ ]:
# The size of the data set was examined. It consists of 1000 observation units and 10 variables.
df.shape

In [ ]:
#Feature information
df.info()

In [ ]:
df.describe([0.10,0.25,0.50,0.75,0.90,0.95,0.99]).T

In [ ]:
# Are there any null values?
df.isnull().values.any()

In [ ]:
# How Many ?
df.isnull().sum()

### 1.2. Categorical Variable

In [ ]:
cat_cols = [col for col in df.columns if df[col].dtypes == 'O']
print(cat_cols,"\n")
print('Number of Categoric Variable: ', len(cat_cols))

for col in cat_cols:
    print(df[col].value_counts(), "\n\n")

### 1.3. Numerical Variables

In [ ]:
num_cols = [col for col in df.columns if df[col].dtypes != 'O' and col not in "Id"]
print(num_cols,"\n")
print('Number of Numerical Variable: ', len(num_cols))

### 1.4. Data Visualization

In [ ]:
trace0 = go.Bar(
            x = df[df["Risk"]== 'good']["Risk"].value_counts().index.values,
            y = df[df["Risk"]== 'good']["Risk"].value_counts().values,
            name='Good credit')

trace1 = go.Bar(
            x = df[df["Risk"]== 'bad']["Risk"].value_counts().index.values,
            y = df[df["Risk"]== 'bad']["Risk"].value_counts().values,
            name='Bad credit')


data = [trace0, trace1]
layout = go.Layout(yaxis=dict(title='Count'),
                   xaxis=dict(title='Risk Variable'),title='Target variable distribution')

fig = go.Figure(data=data, layout=layout)
fig.data[0].marker.line.width = 4
fig.data[0].marker.line.color = "black"
fig.data[1].marker.line.width = 4
fig.data[1].marker.line.color = "black"
py.iplot(fig, filename='grouped-bar')

In [ ]:
df_good = df.loc[df["Risk"] == 'good']['Age'].values.tolist()
df_bad = df.loc[df["Risk"] == 'bad']['Age'].values.tolist()
df_age = df['Age'].values.tolist()

#First plot
trace0 = go.Histogram(x=df_good,histnorm='probability', name="Good Credit")
#Second plot
trace1 = go.Histogram(x=df_bad, histnorm='probability', name="Bad Credit")
#Third plot
trace2 = go.Histogram(x=df_age, histnorm='probability', name="Overall Age")

#Creating the grid
fig = tls.make_subplots(rows=2, cols=2, specs=[[{}, {}], [{'colspan': 2}, None]], 
                        subplot_titles=('Good','Bad', 'General Distribuition'))

#setting the figs
fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)
fig.append_trace(trace2, 2, 1)

fig['layout'].update(showlegend=True, title='Age Distribuition', bargap=0.05)
py.iplot(fig, filename='custom-sized-subplot-with-subplot-titles')

In [ ]:
#See together in graphs accordings to Frequency and Count:
df_good = df[df["Risk"] == 'good']
df_bad = df[df["Risk"] == 'bad']

fig, ax = plt.subplots(nrows=2, figsize=(12,8))
plt.subplots_adjust(hspace = 0.4, top = 0.8)

g1 = sns.distplot(df_good["Age"], ax=ax[0], color="g")
g1 = sns.distplot(df_bad["Age"], ax=ax[0], color='r')
g1.set_title("Age Distribuition", fontsize=15)
g1.set_xlabel("Age")
g1.set_xlabel("Frequency")

g2 = sns.countplot(x="Age",data=df, palette="hls", ax=ax[1], hue = "Risk")
g2.set_title("Age Counting by Risk", fontsize=15)
g2.set_xlabel("Age")
g2.set_xlabel("Count")
plt.show()

In [ ]:
#Let's look the Credit Amount column
interval = (18, 25, 35, 60, 120)

cats = ['Student', 'Young', 'Adult', 'Senior']
df["Age_cat"] = pd.cut(df.Age, interval, labels=cats)

# And let's re-create "df_bad" dataframes with "df_good" we created.
df_good = df[df["Risk"] == 'good']
df_bad = df[df["Risk"] == 'bad']

In [ ]:
trace0 = go.Box(y=df_good["Credit amount"], x=df_good["Age_cat"], name='Good credit', marker=dict(color='LightSkyBlue'))

trace1 = go.Box(y=df_bad['Credit amount'], x=df_bad['Age_cat'], name='Bad credit', marker=dict(color='DarkSlateGrey'))
    
data = [trace0, trace1]

layout = go.Layout(yaxis=dict(title='Credit Amount (Deutsch Mark)',zeroline=False),xaxis=dict(title='Age Categorical'),boxmode='group')
fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='box-age-cat')

In [ ]:
#First plot
trace0 = go.Bar(
                x = df[df["Risk"]== 'good']["Housing"].value_counts().index.values,
                y = df[df["Risk"]== 'good']["Housing"].value_counts().values,
                name='Good credit'
                )

#Second plot
trace1 = go.Bar(
                x = df[df["Risk"]== 'bad']["Housing"].value_counts().index.values,
                y = df[df["Risk"]== 'bad']["Housing"].value_counts().values,
                name="Bad Credit"
                )

data = [trace0, trace1]

layout = go.Layout(title='Housing Distribuition')


fig = go.Figure(data=data, layout=layout)
fig.data[0].marker.line.width = 4
fig.data[0].marker.line.color = "black"
fig.data[1].marker.line.width = 4
fig.data[1].marker.line.color = "black"

py.iplot(fig, filename='Housing-Grouped')
plt.show()

In [ ]:
fig = {
    "data": [{
            "type": 'violin',
            "x": df_good['Housing'],
            "y": df_good['Credit amount'],
            "legendgroup": 'Good Credit',
            "scalegroup": 'No',
            "name": 'Good Credit',
            "side": 'negative',
            "box": {"visible": True},
            "meanline": {"visible": True},
            "line": {"color": '#673D43'}},
            {
            "type": 'violin',
            "x": df_bad['Housing'],
            "y": df_bad['Credit amount'],
            "legendgroup": 'Bad Credit',
            "scalegroup": 'No',
            "name": 'Bad Credit',
            "side": 'positive',
            "box": {"visible": True},
            "meanline": {"visible": True},
            "line": {"color": '#AB9C2B'}
             }],"layout" : { "yaxis": {"zeroline": False,},"violingap": 0,"violinmode": "overlay"}}

py.iplot(fig, filename = 'violin/split', validate = False)

In [ ]:
#First plot
trace0 = go.Bar(x = df[df["Risk"]== 'good']["Sex"].value_counts().index.values,
                y = df[df["Risk"]== 'good']["Sex"].value_counts().values,
                name='Good credit')

#First plot 2
trace1 = go.Bar(x = df[df["Risk"]== 'bad']["Sex"].value_counts().index.values,
                y = df[df["Risk"]== 'bad']["Sex"].value_counts().values,
                name="Bad Credit")

#Second plot
trace2 = go.Box(x = df[df["Risk"]== 'good']["Sex"],
                y = df[df["Risk"]== 'good']["Credit amount"],
                name=trace0.name)

#Second plot 2
trace3 = go.Box(x = df[df["Risk"]== 'bad']["Sex"],
                y = df[df["Risk"]== 'bad']["Credit amount"],
                name=trace1.name)

data = [trace0, trace1, trace2,trace3]

fig = tls.make_subplots(rows=1, cols=2, subplot_titles=('Sex Count', 'Credit Amount by Sex'))

fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 1, 2)

fig['layout'].update(height=400, width=800, title='Sex Distribuition', boxmode='group')
py.iplot(fig, filename='sex-subplot')
plt.show()

In [ ]:
#First plot
trace0 = go.Bar(x = df[df["Risk"]== 'good']["Job"].value_counts().index.values,
                y = df[df["Risk"]== 'good']["Job"].value_counts().values,
                name='Good credit Distribuition')

#Second plot
trace1 = go.Bar(x = df[df["Risk"]== 'bad']["Job"].value_counts().index.values,
                y = df[df["Risk"]== 'bad']["Job"].value_counts().values,
                name="Bad Credit Distribuition")

data = [trace0, trace1]

layout = go.Layout(title='Job Distribuition')

fig = go.Figure(data=data, layout=layout)
fig.data[0].marker.line.width = 4
fig.data[0].marker.line.color = "black"
fig.data[1].marker.line.width = 4
fig.data[1].marker.line.color = "black"
py.iplot(fig, filename='grouped-bar')

In [ ]:
trace0 = go.Box(x=df_good["Job"],
                y=df_good["Credit amount"],
                name='Good credit',
                marker=dict(color='LightSkyBlue'))

trace1 = go.Box(x=df_bad['Job'],
                y=df_bad['Credit amount'],
                name='Bad credit',
                marker=dict(color='DarkSlateGrey'))
    
data = [trace0, trace1]

layout = go.Layout(yaxis=dict(title='Credit Amount distribuition by Job',zeroline=False),
                   xaxis=dict(title='Job'),boxmode='group')
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='box-age-cat')
plt.show()

In [ ]:
fig = {"data": [{
                 "type": 'violin',
                 "x": df_good['Job'],
                 "y": df_good['Age'],
                 "legendgroup": 'Good Credit',
                 "scalegroup": 'No',
                 "name": 'Good Credit',
                 "side": 'negative',
                 "box": {"visible": True},
                 "meanline": {"visible": True},
                 "line": {"color": '#673D43'}
                 },
                 {
                 "type": 'violin',
                 "x": df_bad['Job'],
                 "y": df_bad['Age'],
                 "legendgroup": 'Bad Credit',
                 "scalegroup": 'No',
                 "name": 'Bad Credit',
                 "side": 'positive',
                 "box": {"visible": True},
                 "meanline": {"visible": True},
                 "line": {"color": 'green'}
                 }],
            "layout" : {
            "yaxis": {
            "zeroline": False,},
            "violingap": 0,
            "violinmode": "overlay"}}
py.iplot(fig, filename = 'Age-Housing', validate = False)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12,12), nrows=2)

g1 = sns.boxplot(x="Job", y="Credit amount", data=df, 
            palette="husl", ax=ax[0], hue="Risk")
g1.set_title("Credit Amount by Job", fontsize=15)
g1.set_xlabel("Job Reference", fontsize=12)
g1.set_ylabel("Credit Amount", fontsize=12)

g2 = sns.violinplot(x="Job", y="Age", data=df, ax=ax[1],  
               hue="Risk", split=True, palette="Blues")
g2.set_title("Job Type reference x Age", fontsize=15)
g2.set_xlabel("Job Reference", fontsize=12)
g2.set_ylabel("Age", fontsize=12)

plt.subplots_adjust(hspace = 0.4,top = 0.9)

plt.show()

In [ ]:
# Add histogram data
x1 = np.log(df_good['Credit amount']) 
x2 = np.log(df_bad["Credit amount"])

# Group data together
hist_data = [x1, x2]

group_labels = ['Good Credit', 'Bad Credit']
colors = ['rgb(0, 0, 100)', 'rgb(0, 200, 200)']

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels, bin_size=.1, colors=colors)

# Plot!
py.iplot(fig, filename='Distplot with Multiple Datasets')

In [ ]:
#Ploting the good and bad dataframes in distplot
plt.figure(figsize = (8,5))

g= sns.distplot(df_good['Credit amount'], color='y')
g = sns.distplot(df_bad["Credit amount"], color='g')
g.set_title("Credit Amount Frequency distribuition", fontsize=18)
plt.show()

In [ ]:
count_good = go.Bar(x = df_good["Saving accounts"].value_counts().index.values,
                    y = df_good["Saving accounts"].value_counts().values,
                    name='Good credit')
count_bad = go.Bar(x = df_bad["Saving accounts"].value_counts().index.values,
                   y = df_bad["Saving accounts"].value_counts().values,
                   name='Bad credit')


box_1 = go.Box(x=df_good["Saving accounts"],
               y=df_good["Credit amount"],
               name='Good credit')

box_2 = go.Box(x=df_bad["Saving accounts"],
               y=df_bad["Credit amount"],
               name='Bad credit')

scat_1 = go.Box(x=df_good["Saving accounts"],
                y=df_good["Age"],
                name='Good credit')

scat_2 = go.Box(x=df_bad["Saving accounts"],
                y=df_bad["Age"],
                name='Bad credit')

data = [scat_1, scat_2, box_1, box_2, count_good, count_bad]

fig = tls.make_subplots(rows=2, cols=2, specs=[[{}, {}], [{'colspan': 2}, None]],
                          subplot_titles=('Count Saving Accounts','Credit Amount by Savings Acc', 
                                          'Age by Saving accounts'))

fig.append_trace(count_good, 1, 1)
fig.append_trace(count_bad, 1, 1)

fig.append_trace(box_2, 1, 2)
fig.append_trace(box_1, 1, 2)

fig.append_trace(scat_1, 2, 1)
fig.append_trace(scat_2, 2, 1)

fig['layout'].update(height=700, width=800, title='Saving Accounts Exploration', boxmode='group')

py.iplot(fig, filename='combined-savings')
plt.show()

In [ ]:
print("Values describe: ")
print(pd.crosstab(df.Purpose, df.Risk))

plt.figure(figsize = (14,12))

plt.subplot(221)
g = sns.countplot(x="Purpose", data=df,palette="husl", hue = "Risk")

g.set_xticklabels(g.get_xticklabels(),rotation=45)
g.set_xlabel("", fontsize=12)
g.set_ylabel("Count", fontsize=12)
g.set_title("Purposes Count", fontsize=20)

plt.subplot(222)
g1 = sns.violinplot(x="Purpose", y="Age", data=df, 
                    palette="Blues", hue = "Risk",split=True)
g1.set_xticklabels(g1.get_xticklabels(),rotation=45)
g1.set_xlabel("", fontsize=12)
g1.set_ylabel("Count", fontsize=12)
g1.set_title("Purposes by Age", fontsize=20)

plt.subplot(212)
g2 = sns.boxplot(x="Purpose", y="Credit amount", data=df, 
               palette="Set2", hue = "Risk")
g2.set_xlabel("Purposes", fontsize=12)
g2.set_ylabel("Credit Amount", fontsize=12)
g2.set_title("Credit Amount distribuition by Purposes", fontsize=20)

plt.subplots_adjust(hspace = 0.6, top = 0.8)

plt.show()

In [ ]:
plt.figure(figsize = (12,14))

g= plt.subplot(311)
g = sns.countplot(x="Duration", data=df, 
              palette="husl",  hue = "Risk")
g.set_xlabel("Duration Distribuition", fontsize=12)
g.set_ylabel("Count", fontsize=12)
g.set_title("Duration Count", fontsize=20)

g1 = plt.subplot(312)
g1 = sns.pointplot(x="Duration", y ="Credit amount",data=df,
                   hue="Risk", palette="Set2")
g1.set_xlabel("Duration", fontsize=12)
g1.set_ylabel("Credit Amount(US)", fontsize=12)
g1.set_title("Credit Amount distribuition by Duration", fontsize=20)

g2 = plt.subplot(313)
g2 = sns.distplot(df_good["Duration"], color='y')
g2 = sns.distplot(df_bad["Duration"], color='b')
g2.set_xlabel("Duration", fontsize=12)
g2.set_ylabel("Frequency", fontsize=12)
g2.set_title("Duration Frequency x good and bad Credit", fontsize=20)

plt.subplots_adjust(wspace = 0.4, hspace = 0.4,top = 0.9)

plt.show()

In [ ]:
#First plot
trace0 = go.Bar(
    x = df[df["Risk"]== 'good']["Checking account"].value_counts().index.values,
    y = df[df["Risk"]== 'good']["Checking account"].value_counts().values,
    name='Good credit Distribuition' 
    
)

#Second plot
trace1 = go.Bar(
    x = df[df["Risk"]== 'bad']["Checking account"].value_counts().index.values,
    y = df[df["Risk"]== 'bad']["Checking account"].value_counts().values,
    name="Bad Credit Distribuition"
)

data = [trace0, trace1]

layout = go.Layout(
    title='Checking accounts Distribuition',
    xaxis=dict(title='Checking accounts name'),
    yaxis=dict(title='Count'),
    barmode='group'
)


fig = go.Figure(data=data, layout=layout)
fig.data[0].marker.line.width = 4
fig.data[0].marker.line.color = "black"
fig.data[1].marker.line.width = 4
fig.data[1].marker.line.color = "black"
py.iplot(fig, filename = 'Age-ba', validate = False)

In [ ]:
df_good = df[df["Risk"] == 'good']
df_bad = df[df["Risk"] == 'bad']

trace0 = go.Box(
    y=df_good["Credit amount"],
    x=df_good["Checking account"],
    name='Good credit'
)

trace1 = go.Box(
    y=df_bad['Credit amount'],
    x=df_bad['Checking account'],
    name='Bad credit'
    
)
    
data = [trace0, trace1]

layout = go.Layout(
    yaxis=dict(
        title='Cheking distribuition'
    ),
    boxmode='group'
)
fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='box-age-cat')

In [ ]:
print("Total values of the most missing variable: ")
print(df.groupby("Checking account")["Checking account"].count())

plt.figure(figsize = (12,10))

g = plt.subplot(221)
g = sns.countplot(x="Checking account", data=df,palette="husl", hue="Risk")
g.set_xlabel("Checking Account", fontsize=12)
g.set_ylabel("Count", fontsize=12)
g.set_title("Checking Account Counting by Risk", fontsize=20)

g1 = plt.subplot(222)
g1 = sns.violinplot(x="Checking account", y="Age", data=df, palette="Set2", hue = "Risk",split=True)
g1.set_xlabel("Checking Account", fontsize=12)
g1.set_ylabel("Age", fontsize=12)
g1.set_title("Age by Checking Account", fontsize=20)

g2 = plt.subplot(212)
g2 = sns.boxplot(x="Checking account",y="Credit amount", data=df,hue='Risk',palette="pastel")
g2.set_xlabel("Checking Account", fontsize=12)
g2.set_ylabel("Credit Amount(US)", fontsize=12)
g2.set_title("Credit Amount by Cheking Account", fontsize=20)

plt.subplots_adjust(wspace = 0.2, hspace = 0.3, top = 0.9)

plt.show()

In [ ]:
plt.figure(figsize = (10,6))

g = sns.violinplot(x="Housing",y="Job",data=df,
                   hue="Risk", palette="deep", as_cmap=True, split=True)
g.set_xlabel("Housing", fontsize=12)
g.set_ylabel("Job", fontsize=12)
g.set_title("Housing x Job - Dist", fontsize=20)

plt.show()

In [ ]:
print("Job status by gender", pd.crosstab(df.Sex, df.Job), sep="\n\n")
print("\n\n\nAccount status by gender\n\n", pd.crosstab(df["Checking account"],df.Sex))

### 1.5. Numerical Variable Analysis

In [ ]:
def hist_for_nums(data, numeric_cols):
    col_counter = 0
    data = data.copy()
    for col in numeric_cols:
        data[col].plot.hist(alpha=0.5, color='y')
        plt.xlabel(col)
        plt.title(col)
        plt.show()
        col_counter += 1
    print(col_counter, "variables have been plotted")

hist_for_nums(df, num_cols)

In [ ]:
df_good = df.loc[df["Risk"] == 'good']['Age'].values.tolist()
df_bad = df.loc[df["Risk"] == 'bad']['Age'].values.tolist()
df_age = df['Age'].values.tolist()

#First plot
trace0 = go.Histogram(x=df_good,
                      histnorm='probability',
                      name="Good Credit")
#Second plot
trace1 = go.Histogram(x=df_bad,
                      histnorm='probability',
                      name="Bad Credit")
#Third plot
trace2 = go.Histogram(x=df_age,
                      histnorm='probability',
                      name="Overall Age")

#Creating the grid
fig = tls.make_subplots(rows=2, cols=2, specs=[[{}, {}], [{'colspan': 2}, None]],
                          subplot_titles=('Good','Bad', 'General Distribuition'))

#setting the figs
fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)
fig.append_trace(trace2, 2, 1)

fig['layout'].update(showlegend=True, title='Age Distribuition', bargap=0.05)
py.iplot(fig, filename='custom-sized-subplot-with-subplot-titles')

In [ ]:
df_good = df[df["Risk"] == 'good']
df_bad = df[df["Risk"] == 'bad']

fig, ax = plt.subplots(nrows=2, figsize=(12,8))
plt.subplots_adjust(hspace = 0.4, top = 0.8)

g1 = sns.distplot(df_good["Age"], ax=ax[0], color="g")
g1 = sns.distplot(df_bad["Age"] , ax=ax[0], color='r')
g1.set_title("Age Distribuition", fontsize=15)
g1.set_xlabel("Age")
g1.set_xlabel("Frequency")

g2 = sns.countplot(x="Age",data=df, palette="hls", ax=ax[1], hue = "Risk")

g2.set_title("Age Counting by Risk", fontsize=15)
g2.set_xlabel("Age")
g2.set_xlabel("Count")
plt.show()

In [ ]:
#Let's look the Credit Amount column
interval = (18, 25, 35, 60, 80)

cats = ['Student', 'Young', 'Adult', 'Senior']
df["Age_cat"] = pd.cut(df.Age, interval, labels=cats)
df.head()

In [ ]:
# And let's re-create "df_bad" dataframes with "df_good" we created.
df_good = df[df["Risk"] == 'good']
df_bad = df[df["Risk"] == 'bad']

In [ ]:
#First plot
trace0 = go.Bar(x = df[df["Risk"]== 'good']["Housing"].value_counts().index.values,
                y = df[df["Risk"]== 'good']["Housing"].value_counts().values,
                name='Good credit')

#Second plot
trace1 = go.Bar(x = df[df["Risk"]== 'bad']["Housing"].value_counts().index.values,
                y = df[df["Risk"]== 'bad']["Housing"].value_counts().values,
                name="Bad Credit")

data = [trace0, trace1]

layout = go.Layout(title='Housing Distribuition')

fig = go.Figure(data=data, layout=layout)
fig.data[0].marker.line.width = 4
fig.data[0].marker.line.color = "black"
fig.data[1].marker.line.width = 4
fig.data[1].marker.line.color = "black"

py.iplot(fig, filename='Housing-Grouped')

In [ ]:
#First plot
trace0 = go.Bar(x = df[df["Risk"]== 'good']["Sex"].value_counts().index.values,
                y = df[df["Risk"]== 'good']["Sex"].value_counts().values,
                name='Good credit')

#First plot 2
trace1 = go.Bar(x = df[df["Risk"]== 'bad']["Sex"].value_counts().index.values,
                y = df[df["Risk"]== 'bad']["Sex"].value_counts().values,
                name="Bad Credit")

data = [trace0, trace1]

layout = go.Layout(title='Sex Distribuition')

fig = go.Figure(data=data, layout=layout)
fig.data[0].marker.line.width = 4
fig.data[0].marker.line.color = "black"
fig.data[1].marker.line.width = 4
fig.data[1].marker.line.color = "black"

py.iplot(fig, filename='Sex_Distribuition')

In [ ]:
#First plot
trace0 = go.Bar(x = df[df["Risk"]== 'good']["Job"].value_counts().index.values,
                y = df[df["Risk"]== 'good']["Job"].value_counts().values,
                name='Good credit Distribuition')

#Second plot
trace1 = go.Bar(x = df[df["Risk"]== 'bad']["Job"].value_counts().index.values,
                y = df[df["Risk"]== 'bad']["Job"].value_counts().values,
                name="Bad Credit Distribuition")

data = [trace0, trace1]

layout = go.Layout(title='Job Distribuition')

fig = go.Figure(data=data, layout=layout)
fig.data[0].marker.line.width = 4
fig.data[0].marker.line.color = "black"
fig.data[1].marker.line.width = 4
fig.data[1].marker.line.color = "black"
py.iplot(fig, filename='grouped-bar')

In [ ]:
print("Description of Distribuition Saving accounts by Risk:  ")
print(pd.crosstab(df["Saving accounts"],df.Risk))

fig, ax = plt.subplots(3,1, figsize=(12,12))
g = sns.countplot(x="Saving accounts", data=df, palette="husl", ax=ax[0],hue="Risk")
g.set_title("Saving Accounts Count", fontsize=15)
g.set_xlabel("Saving Accounts type", fontsize=12)
g.set_ylabel("Count", fontsize=12)

g1 = sns.violinplot(x="Saving accounts", y="Job", data=df, palette="Blues", hue = "Risk", ax=ax[1],split=True)
g1.set_title("Saving Accounts by Job", fontsize=15)
g1.set_xlabel("Savings Accounts type", fontsize=12)
g1.set_ylabel("Job", fontsize=12)

g = sns.boxplot(x="Saving accounts", y="Credit amount", data=df, ax=ax[2], hue = "Risk",palette="pastel")
g2.set_title("Saving Accounts by Credit Amount", fontsize=15)
g2.set_xlabel("Savings Accounts type", fontsize=12)
g2.set_ylabel("Credit Amount(US)", fontsize=12)

plt.subplots_adjust(hspace = 0.4,top = 0.9)

plt.show()

### All Functions

In [ ]:
def missing_values_table(dataframe):
    variables_with_na = [col for col in dataframe.columns if dataframe[col].isnull().sum() > 0]
    n_miss = dataframe[variables_with_na].isnull().sum().sort_values(ascending=False)
    ratio = (dataframe[variables_with_na].isnull().sum() / dataframe.shape[0] * 100).sort_values(ascending=False)
    missing_df = pd.concat([n_miss, np.round(ratio, 2)], axis=1, keys=['n_miss', 'ratio'])
    print(missing_df)
    return variables_with_na
##################################################################################################################
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.05)
    quartile3 = dataframe[variable].quantile(0.95)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit
##################################################################################################################
def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit
##################################################################################################################
def has_outliers(dataframe, num_col_names, plot=False):
    variable_names = []
    for col in num_col_names:
        low_limit, up_limit = outlier_thresholds(dataframe, col)
        if dataframe[(dataframe[col] > up_limit) | (dataframe[col] < low_limit)].any(axis=None):
            number_of_outliers = dataframe[(dataframe[col] > up_limit) | (dataframe[col] < low_limit)].shape[0]
            print(col, ":", number_of_outliers)
            variable_names.append(col)
            if plot:
                sns.boxplot(x=dataframe[col])
                plt.show()
    return variable_names
##################################################################################################################
def rare_analyser(dataframe, categorical_columns, target, rare_perc):
    rare_columns = [col for col in categorical_columns
                    if (dataframe[col].value_counts() / len(dataframe) < rare_perc).any(axis=None)]
    for var in rare_columns:
        print(var, " : ", len(dataframe[var].value_counts()))

        print(pd.DataFrame({"COUNT": dataframe[var].value_counts(),
                            "RATIO": dataframe[var].value_counts() / len(dataframe),
                            "TARGET_MEAN": dataframe.groupby(var)[target].mean(),
                            "TARGET_MEDIAN": dataframe.groupby(var)[target].median()}),
              end="\n\n\n")

    print("There are",len(rare_columns), "variables with rare classes.")
##################################################################################################################    
def label_encoder(dataframe, categorical_columns):
    labelencoder = preprocessing.LabelEncoder()
    for col in categorical_columns:
        if dataframe[col].nunique() == 2:
            dataframe[col] = labelencoder.fit_transform(dataframe[col])
    return dataframe
################################################################################################################## 
def one_hot_encoder(dataframe, categorical_columns, nan_as_category=False):
    original_columns = list(dataframe.columns)
    dataframe = pd.get_dummies(dataframe, columns=categorical_columns,
                               dummy_na=nan_as_category, drop_first=True)
    new_columns = [col for col in dataframe.columns if col not in original_columns]
    return dataframe, new_columns

In [ ]:
df = pd.read_csv('../input/german-credit-data-with-risk/german_credit_data.csv', index_col=0)
df.head()

# 2. Data Preprocessing

In [ ]:
num_cols = [col for col in df.columns if df[col].dtypes != 'O' and col not in "Id"]
print(num_cols,"\n")
print('Number of Numerical Variable: ', len(num_cols))

In [ ]:
cat_cols = [col for col in df.columns if df[col].dtypes == 'O']
print(cat_cols,"\n")
print('Number of Categoric Variable: ', len(cat_cols))

## 2.1. Missing Value Analysis

In [ ]:
missing_values_table(df)

In [ ]:
interval = (18, 25, 35, 60, 120)
cats = ['Student', 'Young', 'Adult', 'Senior']
df["Age_cat"] = pd.cut(df.Age, interval, labels=cats)

df["Saving accounts"] = df.groupby(["Sex", "Risk", "Age_cat"])["Saving accounts"].transform(
    lambda x: x.fillna(x.mode()[0]))

# Let's try to fill in the missing values ​​logically by taking groupby.
df["Checking account"] = df.groupby(["Sex", "Risk", "Age_cat"])["Checking account"].transform(
    lambda x: x.fillna(x.mode()[0]))

In [ ]:
missing_values_table(df)

## 2.2. Outlies Analysis

In [ ]:
has_outliers(df, ["Age", "Credit amount", "Duration"])

## 2.3. Rare Analysis

In [ ]:
label_columns = ["Sex", "Risk"]
label_encoder(df, label_columns)

In [ ]:
temp_cat = ["Sex", "Job", "Housing", "Saving accounts", "Checking account", "Duration", "Purpose"]
rare_analyser(df, temp_cat, "Risk", 0.5)

In [ ]:
df.loc[df["Duration"] == 5, ["Duration"]] = 4
df.loc[df["Duration"] == 7, ["Duration"]] = 4
df.loc[df["Duration"] == 11, ["Duration"]] = 4

df.loc[df["Duration"] == 8, ["Duration"]] = 6
df.loc[df["Duration"] == 10, ["Duration"]] = 6

df.loc[df["Duration"] == 12, ["Duration"]] = 9

df.loc[df["Duration"] == 24, ["Duration"]] = 21

df.loc[df["Duration"] == 28, ["Duration"]] = 27
df.loc[df["Duration"] == 30, ["Duration"]] = 27
df.loc[df["Duration"] == 33, ["Duration"]] = 27

df.loc[df["Duration"] == 42, ["Duration"]] = 39

df.loc[df["Purpose"] == "domestic appliances", ["Purpose"]] = "furniture/equipment"

rare_analyser(df, temp_cat, "Risk", 0.5)

# 3.Feature Engineering

In [ ]:
# Deriving Year variable from Duration variable.
# Thus, it will be understood how many years of customers there are.
df["Year"] = str(df["Duration"])
df.loc[df["Duration"] <= 12, "Year"] = "0-1 year"
df.loc[(df["Duration"] > 12) & (df["Duration"] <= 24), "Year"] = "1-2 year"
df.loc[(df["Duration"] > 24) & (df["Duration"] <= 36), "Year"] = "2-3 year"
df.loc[(df["Duration"] > 36) & (df["Duration"] <= 48), "Year"] = "3-4 year"
df.loc[(df["Duration"] > 48) & (df["Duration"] <= 60), "Year"] = "4-5 year"
df.loc[(df["Duration"] > 60) & (df["Duration"] <= 72), "Year"] = "5-6 year"
df.loc[(df["Duration"] > 72) & (df["Duration"] <= 84), "Year"] = "6-7 year"

df["Status"] = pd.qcut(df["Credit amount"], 4, labels=["poor", "mid", "upper", "rich"])
df.head()

In [ ]:
num_cols = [col for col in df.columns if df[col].dtypes != 'O']
print(num_cols)

In [ ]:
num_cols = [col for col in df.columns if df[col].dtypes != 'O' and
            col not in "Risk" and
            col not in "more_cat_cols"]
num_cols

In [ ]:
has_outliers(df, ['Age', 'Sex', 'Job','Duration'])

## 3.1. Label Encoding

In [ ]:
one_hot_columns = ["Job","Housing", "Saving accounts", "Checking account", "Purpose", "Age_cat", "Year","Status"]
df, one_hot_encodeds = one_hot_encoder(df, one_hot_columns)
df.head()

In [ ]:
like_num = [col for col in df.columns if df[col].dtypes != 'O' and len(df[col].value_counts()) < 20]


cols_need_scale = [col for col in df.columns if col not in "Risk"
                   and col not in "Age"
                   and col not in "Credit amount"
                   and col not in "Duration"
                   and col not in like_num]


def robust_scaler(variable):
    var_median = variable.median()
    quartile1 = variable.quantile(0.05)
    quartile3 = variable.quantile(0.95)
    interquantile_range = quartile3 - quartile1
    if int(interquantile_range) == 0:
        quartile1 = variable.quantile(0.05)
        quartile3 = variable.quantile(0.95)
        interquantile_range = quartile3 - quartile1
        z = (variable - var_median) / interquantile_range
        return round(z, 3)
    else:
        z = (variable - var_median) / interquantile_range
    return round(z, 3)


for col in cols_need_scale:
    df[col] = robust_scaler(df[col])
    
df.drop("Age", axis=1, inplace = True)    
df.head()

# 4.Model

In [ ]:
X = df.drop("Risk", axis=1)
y = np.ravel(df[["Risk"]])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = 357)

In [ ]:
X_train.head()

In [ ]:
# Let's set the parameters to find the best parameter to use in our models. These parameters will be chosen as the best to visit with GridSearchCV.
rf_params = {"max_depth": [3, 5, 8],
             "max_features": [8, 15, 25],
             "n_estimators": [200, 500, 1000],
             "min_samples_split": [2, 5, 10]}

lgbm_params = {"learning_rate": [0.01, 0.1],
               "n_estimators": [200, 500, 1000],
               "max_depth": [3, 5, 8],
               "colsample_bytree": [1, 0.8, 0.5]}

rf = RandomForestClassifier(random_state=357)
lgbm = LGBMClassifier(random_state=357)

# Let's try the parameters one by one with the help of GridSearchCV and find the best parameter.
gs_cv_rf = GridSearchCV(rf, rf_params, cv=10, n_jobs=-1, verbose=2).fit(X_train, y_train)
gs_cv_lgbm = GridSearchCV(lgbm, lgbm_params, cv=10, n_jobs=-1, verbose=2).fit(X_train, y_train)

In [ ]:
ann = Sequential()
ann.add(Dense(X.shape[1], activation='relu', input_dim = X.shape[1]))
ann.add(Dense(254, activation='relu'))
ann.add(Dense(1, activation = 'sigmoid'))
ann.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['accuracy'])

ann_tuned = ann.fit(X_train, np.array(y_train), batch_size = 10, epochs = 10, verbose = 0)
knn_tuned = KNeighborsClassifier().fit(X_train, y_train)
svm_tuned = SVC(random_state=357,gamma='auto').fit(X_train, y_train)
gnb_tuned = GaussianNB().fit(X_train, y_train)
dtc_tuned = DecisionTreeClassifier(random_state=357).fit(X_train, y_train)

# Let's tune the models with the best parameters found.
rf_tuned = RandomForestClassifier(**gs_cv_rf.best_params_, random_state=357).fit(X_train, y_train)
lgbm_tuned = LGBMClassifier(**gs_cv_lgbm.best_params_, random_state=357).fit(X_train, y_train)

In [ ]:
# Result
models = [("ANN", ann_tuned),
          ("KNN", knn_tuned),
          ("SVM", svm_tuned),
          ("GNB", gnb_tuned),
          ("DTC", dtc_tuned),
          ("RF", rf_tuned),
          ("LGBM", lgbm_tuned)]

for name, model in models:
    if name != "ANN":
        y_pred = model.predict(X_test)
        ann.evaluate(X_test, np.array(y_test),verbose = 0)
    else:
        y_pred = ann.predict_classes(X_test)
    acc = accuracy_score(y_test, y_pred)
    msg = "%s: (%f)" % (name, acc)
    print(msg)